In [ ]:
%pip install transformers

In [2]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

import warnings
warnings.filterwarnings('ignore')

import transformers

from transformers import GPT2Tokenizer

import textwrap

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [4]:
with open('IT_Advices_Proverbs.txt', encoding='utf8') as f:
    text = f.read()
    text = text.split('\n')

random.shuffle(text)
print(text)


['Если назначение элемента или сочетания неясно, то его поведение должно быть наиболее ожидаемым со стороны пользователя.', 'Обобщение частных случаев — безосновательный перенос характеристик частных или даже единичных случаев на обширные их совокупности. Существует множество видов этого когнитивного искажения, классический вариант — теория заговора.', 'Эффект первенства — тенденция переоценивать изначальные события более, чем последующие события.', 'Эффект телескопа — этот эффект состоит в том, что недавние события кажутся более отдалёнными, а более дальние события — более близкими во времени.', 'Амплификация — вложение в достижение цели больше усилий, чем необходимо, попытка «убить муху кувалдой». Например, чрезмерно детальное планирование в условиях отсутствия в достаточном объёме исходных данных и наличия сильно влияющих на результат неопределённых или случайных факторов.', 'Сначала придумайте решение задачи, потом напишите код. ', 'Авось да небось — родные братья, оба лежни', 'Эфф

In [5]:
train = []
test = []
max_length = 0
index_max_length = 0

for idx, value in enumerate(text):
  tokens = tokenizer.encode(value, add_special_tokens=True)
  tokens = np.array(tokens)

  curr_len = len(tokens)
  if curr_len >= max_length:
    max_length = curr_len
    index_max_length = idx

  if idx <= (len(text) * .8):
    train.append(tokens)

  else:
    test.append(tokens)

train = np.array(train)
test = np.array(test)

print('len(train), len(test): ', len(train), len(test))
print('max_length, index_max_length: ', max_length, index_max_length)


len(train), len(test):  286 71
max_length, index_max_length:  113 25


In [6]:
def Padding(review_int, seq_len):
    '''
    Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(review_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            # print(len(zeros))
            # print(review.shape)
            # new = np.array(zeros) + review
            new = np.append(zeros, review)

        features[i, :] = np.array(new)
            
    return features

train = Padding(train, max_length)
test = Padding(test, max_length)

print(train.shape)
print(train[105, :])

(286, 113)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0   682 22105 22978
   343 33058   746   510 27749 19831 16021   411    17   431 33058   746
    16   828 16021   411    17   431  3193    16 13412  2701   885   542
 18051   282 33058  1290    18]


In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

In [9]:
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

batch_size = 8
epochs = 100

n_train = len(train)//batch_size + 1
n_test = len(test)//batch_size + 1
print(n_train, n_test)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

36 9


In [11]:
def prep_tensors(x, i, batch_size=batch_size):
    start_idx = i*batch_size
    end_idx = start_idx + batch_size
    batch_ids = x[start_idx: end_idx]
    batch_ids = torch.tensor(batch_ids).to(device)
    return torch.tensor(batch_ids).to(device)

preped = prep_tensors(train, 17)
print('preped shape: ', preped.shape)

preped shape:  torch.Size([8, 113])


In [ ]:

for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()

    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)

In [59]:
# Model without training

prompt = 'Дизайн должен быть'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=40,
    num_beams=5,
    do_sample=True,
    temperature=.7,
    top_k=10,
    top_p=0.95,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
  wraped = textwrap.fill(tokenizer.decode(out_), 120)
  wraped = wraped.replace("  ", " ")
  arr = wraped.split('.')
  arr.pop()
  final_out_text = '.'.join(arr) + '.'
  print(final_out_text, end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дизайн должен быть продуман до мелочей. Если вы хотите, чтобы ваш дом выглядел стильно и современно, то вам стоит
обратиться к профессионалам, которые помогут вам в этом.
------------------
Дизайн должен быть не только красивым, но и функциональным.
------------------
Дизайн должен быть продуман до мелочей. Если вы хотите, чтобы ваш дом выглядел стильно и современно, то вам следует
обратить внимание на мебель, которая будет соответствовать стилю вашего дома.
------------------
Дизайн должен быть не только красивым, но и функциональным. В первую очередь, это касается мебели, которая должна быть
удобной, функциональной и практичной.
------------------
Дизайн должен быть продуман до мелочей. Если вы хотите, чтобы ваш дом был уютным и комфортным, то лучше всего
обратиться к профессионалам.
------------------
Дизайн должен быть не только красивым, но и функциональным. В первую очередь, это касается мебели, которая должна быть
функциональной, удобной и практичной.
------------------
Дизайн д

In [60]:
# Trained model

prompt = 'Дизайн должен быть'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)

out = model.generate(
    input_ids=prompt,
    max_length=40,
    num_beams=5,
    do_sample=True,
    temperature=.7,
    top_k=10,
    top_p=0.95,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()

for out_ in out:
  wraped = textwrap.fill(tokenizer.decode(out_), 120)
  # print(wraped)
  wraped = wraped.replace("  ", " ")
  if '.' in wraped:
    arr = wraped.split('.')
    arr.pop()
    final_out_text = '.'.join(arr) + '.'
  else:
    final_out_text = wraped
  print(final_out_text, end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми и в то же время достаточно
детальными, чтобы их можно было приложить к очень большому числу людей.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми в использовании, а не усложнять
реализацию или преуменьшать их.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми в использовании, а не усложнять
реализацию или переносить их в область возможного применения.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми в использовании, а не усложнять
реализацию или преуменьшать их.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми и в то же время достаточно
детальными, чтобы их можно было приложить к очень большому числу людей.
------------------
Дизайн должен быть правильным во всех видимых проя

In [51]:
def generate(prompt, len_gen=20, temperature=.7):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [ ]:
prompt = 'Функция должна быть'
print(textwrap.fill(generate(prompt, 200, temperature=.8), 120))